> 2022.12.16 Ssu-Yun Wang<br/>
[Github @angel870326](https://github.com/angel870326)

# **Manufacturing Data Science Final Project Web**




In [1]:
# sConnect to the Google Drive
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [8]:
import pandas as pd
import numpy as np

## **Installation & Setup (Flask & Ngrok)**

In [2]:
!pip install flask
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Get your AuthToken from ngrok using this link: https://dashboard.ngrok.com/get-started/your-authtoken

In [3]:
!pip install pyngrok==4.1.1
!ngrok authtoken '1qCRUUcFgnhlSf0o5xS5eilRUUk_7mexMJwXSwcCcPEuvCs2q' # your authtoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15982 sha256=4133b4195151df6189bd47b2ff4569ce153004be95ccd6561dddf622cb30a13b
  Stored in directory: /root/.cache/pip/wheels/5e/0a/51/8cb053ccd84481dd3233eba4cdb608bc7a885fd8ca418c0806
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Bootstrap (optional)

In [4]:
!pip install bootstrap-flask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.6 MB 9.7 MB/s 
     |████████████████████████████████| 136 kB 57.5 MB/s 


## **Load Machine Learning Model**

In [ ]:
# import pickle
# model = pickle.load(open('model.pkl', 'rb', ))

## **Features to be adjusted**

In [9]:
# feature_list = [[2, 'Starch Flow'],
#                 [3, 'Amina Flow'],
#                 [5, 'Ore Pulp pH'],
#                 [6, 'Ore Pulp Density'],
#                 [7, 'Flotation Column 01 Air Flow'],
#                 [12, 'Flotation Column 06 Air Flow'],
#                 [16, 'Flotation Column 03 Level'],
#                 [17, 'Flotation Column 04 Level'],
#                 [18, 'Flotation Column 05 Level'],
#                 [20, 'Flotation Column 07 Level']]

In [17]:
feature_list = [2, 3, 5, 6, 7, 12, 16, 17, 18, 20]

In [18]:
labels = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]

## **Run the Web**

In [5]:
from flask_ngrok import run_with_ngrok
from flask_bootstrap import Bootstrap5
from flask import Flask, render_template, redirect, request, session, send_file, url_for, Response, stream_with_context
import pandas as pd
import os
from werkzeug.utils import secure_filename
import json

In [25]:
template_path = '/content/gdrive/MyDrive/碩二上/製造數據科學/MDS_Final_Project_Web_v4/templates'
static_path = '/content/gdrive/MyDrive/碩二上/製造數據科學/MDS_Final_Project_Web_v4/static'

# Downlaod file
download_path = os.path.join(static_path, 'downloads')

# Upload file
upload_folder = os.path.join(static_path, 'uploads') # Define folder to save uploaded files to process further
ALLOWED_EXTENSIONS = {'csv'} # Define allowed files

app = Flask(__name__, template_folder = template_path, static_folder=static_path)
app.config['UPLOAD_FOLDER'] = upload_folder # Configure upload file path flask
app.secret_key = 'MDS Final Project Group 4' # Define secret key to enable session

bootstrap = Bootstrap5(app)
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route("/", methods=['GET'])
def home():
    return render_template('index.html')

@app.route("/download")
def downloadFile():
    download_path = os.path.join(static_path, 'downloads/template.csv')
    return send_file(download_path, as_attachment=True)
 
@app.route('/', methods=['GET', 'POST'])
def uploadFile():
    if request.method == 'POST':
        # upload file flask
        uploaded_df = request.files['uploaded-file']
        # Extracting uploaded data file name
        data_filename = secure_filename(uploaded_df.filename)
        # flask upload file to database (defined uploaded folder in static path)
        uploaded_df.save(os.path.join(app.config['UPLOAD_FOLDER'], data_filename))
        # Storing uploaded file path in flask session
        session['uploaded_data_file_path'] = os.path.join(app.config['UPLOAD_FOLDER'], data_filename)
        return render_template('index.html')

@app.route('/predict', methods=['GET', 'POST'])
def predict():

    # model
    # labels = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
    values = [0, 20, 50, 15, 10, 20, 15, 25, 20, 30]
    results_dict = {"labels": labels, "values": values}

    anomaly = "Anomaly"

    return render_template('predict.html', results_dict=json.dumps(results_dict), anomaly=anomaly)

@app.route('/adjust', methods=['GET', 'POST'])
def adjust():
    # Get slider values
    slider_value = []
    for i in range(1,11):
        slider_value.append(request.form.get('slider{}'.format(i)))

    adjusted_features = {"f1":slider_value[0], "f2":slider_value[1], "f3":slider_value[2], "f4":slider_value[3], "f5":slider_value[4], "f6":slider_value[5], "f7":slider_value[6], "f8":slider_value[7], "f9":slider_value[8], "f10":slider_value[9]}
    print(slider_value)

    # labels = ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10"]
    values = [0, 20, 50, 15, 10, 20, 15, 25, 20, 30]
    values_adj = [0, 20, 50, 15, 10, 20, 15, 25, 20, 10]
    results_dict = {"labels": labels, "values": values, "values_adj": values_adj}

    return render_template('adjust.html', results_dict=json.dumps(results_dict), adjusted_features=adjusted_features)

@app.route('/readjust', methods=['GET'])
def readjust():
    return redirect('predict')


# def predict():
#   input_values = [float(x) for x in request.form.values()]
#   inp_features = [input_values]
  # prediction = model.predict(inp_features)
  # if prediction==1:
  #   return render_template('index.html', prediction_text='?')
  # else:
  #   return render_template('index.html', prediction_text='?')

  
 
if __name__=='__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://b943-35-247-107-199.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:23:17] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:23:18] "GET /static/css/styles.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:23:18] "GET /static/js/scripts.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:23:24] "GET /static/assets/img/bg-masthead.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:23:29] "GET /static/assets/favicon.ico HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:23:35] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:23:37] "GET /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:23:37] "GET /static/js/Chart.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:23:37] "GET /static/js/predict.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:23:53] "POST /adjust HTTP/1.1" 200 -


['0', '0', '0', '0', '0', '0', '0', '0', '0', '0']


INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:23:53] "GET /static/js/adjust.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:23:59] "GET /readjust HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:23:59] "GET /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:24:05] "POST /adjust HTTP/1.1" 200 -


['0', '67', '-56', '0', '0', '0', '0', '67', '0', '0']


INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:24:57] "GET /readjust HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:24:57] "GET /predict HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Dec/2022 15:26:15] "POST /adjust HTTP/1.1" 200 -


['58', '-44', '100', '-44', '0', '-100', '45', '0', '-54', '60']


In [ ]:
# @app.route('/', methods=['GET', "POST"])
# def predict():
#   input_values = [float(x) for x in request.form.values()]
#   inp_features = [input_values]
#   prediction = model.predict(inp_features)
#   if prediction==1:
#     return render_template('index.html', prediction_text='Death event took Place. Person is no more')
#   else:
#     return render_template('index.html', prediction_text='Death event not took Place. Person is alive')
